# Cel biznesowy
### Optymalizacja rozmieszczenia dostępnych pojazdów w celu minimalizacji czasu oczekiwania klientów na przejazd.  
Dzięki identyfikacji kluczowych lokalizacji, w których zapotrzebowanie na przejazdy jest największe, możliwe jest efektywniejsze wykorzystanie floty, zwiększenie liczby realizowanych kursów oraz poprawa satysfakcji klientów. Lepsze dopasowanie podaży do popytu pozwala także na zwiększenie przychodów oraz redukcję kosztów operacyjnych związanych z pustymi przejazdami i nieefektywnym czasem postoju kierowców.

![alt text](trash/merged_data_maps.gif)

# Dane

Dane wykorzystane w analizie pochodzą ze zbiorów NYC Taxi and Limousine Commission (TLC). Dostarczane są one przez: 
- autoryzowanych dostawców technologii w ramach programu Taxicab & Livery Passenger Enhancement Programs (TPEP/LPEP) w przypadku żółtych oraz zielonych taksówek;
- przez firmy oferujące przejazdy na żądanie (np. Lyft, Uber) w przypadku For-Hire Vehicle (“FHV”).

Dane są ustrukturyzowane i udostępniane w formie plików parquet.Do analizy wybrałem dane z 2024 roku ponieważ, są to najświeższe dane z pełnego roku w którym nie obowiązują w mieście ograniczenia związane z COVID-19.

### Metody analizy

Do wykonania analizy wybrałem regresję liniową oraz bardziej złożone modele: Random Forest Regressor i XGBoost Regressor. Wybrałem je ze względu na ich zdolność do modelowania nieliniowych zależności oraz wysoką odporność na przeuczenie.

### Analiza jakości danych źródłowych

Zaobserowane przeze mnie  problemy związane z jakością danych źródłowych to wartości brakujące, kursy których godzina zakończenia jest wcześniejsza niż rozpoczęcia, kursy krótsze niż 0 mil, kursy przewożące ujemną liczbę pasażerów. ID stref większe niż 263 (nieistniejące strefy)

### Przetwarzanie danych

Przetwarzanie danych obejmuje stworzenie kolumn pozwalających na agregację na różnych poziomach czasowych:
- dzień
- miesiąc
- godzina  

Jak również obliczenie ilości przejazdów rozpoczętych z konkretnych stref z podziałem na dni i godziny.

Przetworzone dane zapisałem w bazie mysql by uniknąć konieczności ciagłego przetwarzania ich od nowa.

# Budowa modelu predykcyjnego

Wczytanie danych

In [ ]:
import pandas as pd
from sqlalchemy import create_engine
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV
import numpy as np

engine = create_engine('mysql+pymysql://[]user]:[pass]@[host]:[port]/[scheme]', echo=False)

df = pd.read_sql("SELECT * FROM rides", con=engine)



### Definicja zmiennych wejściowych i wyjściowych,

Zmienne wejściowe:

- Dzień tygodnia (0 – poniedziałek, 6 – niedziela)
- Godzina dnia (0 – 23)
- ID strefy (PULocationID) – numer dzielnicy taksówkowej

Zmienna wyjściowa:

- Liczba przejazdów w danej godzinie i strefie "ride_count"

In [2]:
features = ['hour', 'day_of_week', 'PULocationID']
target = 'ride_count'

X = df[features]
y = df[target]

### Metryki oceny jakości modeli,

Mean Absolute Error (MAE) – średnia różnica między przewidywaną a rzeczywistą liczbą kursów  
Mean Squared Error (MSE) / RMSE – kara za duże błędy, RMSE podkreśla większe różnice  
R² Score – miara dopasowania modelu

### Podział danych
Dane zostały podzielone na dane treningowe (70%) oraz zbiór testowy (15%) i walidacyjny (15%) do dostrajania hiperparametrów

In [3]:
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

### Trenowanie

In [5]:
# Scaling the features for better performance (only for linear regression)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Linear Regression
lr = LinearRegression()
lr.fit(X_train_scaled, y_train)
y_pred_lr = lr.predict(X_test_scaled)

# Random Forest Regressor
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

# XGBoost Regressor
xgb_reg = XGBRegressor(objective="reg:squarederror", n_estimators=100, learning_rate=0.1, random_state=42)
xgb_reg.fit(X_train, y_train)
y_pred_xgb = xgb_reg.predict(X_test)

# Evaluation
def evaluate_model(model_name, y_test, y_pred):
    print(f"{model_name} Model Performance:")
    print(f"  - MAE: {mean_absolute_error(y_test, y_pred):.4f}")
    print(f"  - RMSE: {np.sqrt(mean_squared_error(y_test, y_pred)):.4f}")
    print(f"  - R² Score: {r2_score(y_test, y_pred):.4f}")

evaluate_model("Linear Regression", y_test, y_pred_lr)
evaluate_model("Random Forest", y_test, y_pred_rf)
evaluate_model("XGBoost", y_test, y_pred_xgb)

Linear Regression Model Performance:
  - MAE: 94.4433
  - RMSE: 141.6187
  - R² Score: 0.0635
Random Forest Model Performance:
  - MAE: 17.1920
  - RMSE: 37.3758
  - R² Score: 0.9348
XGBoost Model Performance:
  - MAE: 42.7533
  - RMSE: 69.9432
  - R² Score: 0.7716


### Dostrajanie hiperparametrów modelu XGBoost

In [ ]:
xgb = XGBRegressor(objective="reg:squarederror", random_state=42)

param_grid = {
    'n_estimators': [100, 300, 500],  
    'max_depth': [3, 6, 9],  
    'learning_rate': [0.01, 0.05, 0.1],  
    'subsample': [0.8, 1.0],  
    'colsample_bytree': [0.8, 1.0]  
}

grid_search = GridSearchCV(
    estimator=xgb,
    param_grid=param_grid,
    scoring='neg_mean_squared_error',
    cv=5,
    verbose=2,
    n_jobs=-1
)

grid_search.fit(X_train, y_train)

print("Najlepsze parametry:", grid_search.best_params_)

best_xgb = grid_search.best_estimator_
y_pred = best_xgb.predict(X_test)

print(f"MAE: {mean_absolute_error(y_test, y_pred):.4f}")
print(f"RMSE: {np.sqrt(mean_squared_error(y_test, y_pred)):.4f}")
print(f"R² Score: {r2_score(y_test, y_pred):.4f}")


### Rezultaty

| Model                  | MAE     | RMSE    | R² Score |
|------------------------|---------|---------|----------|
| Linear Regression       | 94.4433 | 141.6187 | 0.0635   |
| Random Forest           | 17.1920 | 37.3758  | 0.9348   |
| XGBoost (before tuning) | 42.7533 | 69.9432  | 0.7716   |
| XGBoost (after tuning)  | 19.0686 | 40.1113  | 0.9249   |

Najgorzej z modelowaniem problemu poradził sobie prosty model regresji liniowej. Las losowy uzyskał najlepsze wyniki wskazuje na to wysoka wartość współczyninika determinacji R^2. Podobną wartość uzyskał model XGBoost po przeprowadzeniu dostrajania hiperparametrów.

# Podsumowanie

Dzięki zastosowaniu modeli regresyjnych możliwe jest zidentyfikowanie stref z największą ilością spodziewanych potencjalnych klientów. Modele takie mogą pozwolić kierowcom taksówek na skrócenie czasu oczekiwania na kolejnego klienta. 

![alt text](image.png)

Przewidywana liczba przejazdów dla poszczególnych stref w środę o godzinie 14-stej.